In [145]:
import pathlib
import pandas as pd
import sys
import arcpy
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# get local path directory
localpath = pathlib.Path.cwd()
module_path = localpath.parent / '2023'
# # adding Scripts/2023 to system path to find utils.py
sys.path.insert(0, str(module_path))
from utils import *

In [49]:
# get 2010 Ecobject data
def get_ecobject_2010_data():
    # make sql database connection with pyodbc
    engine = get_conn('sde_tabular')
    # get dataframe from BMP SQL Database
    with engine.begin() as conn:
        # create dataframe from sql query
        df = pd.read_sql("SELECT * FROM sde_tabular.SDE.Vegetation_EcObject_2010", conn)
    return df
# get identity data of 2010 Ecobject data overlayed with Caldor Burn Severity
def get_ecobject_caldor_identity_data():
    # make sql database connection with pyodbc
    engine = get_conn('sde_tabular')
    # get dataframe from BMP SQL Database
    with engine.begin() as conn:
        # create dataframe from sql query
        df = pd.read_sql("SELECT * FROM sde_tabular.SDE.ThresholdEvaluation_Vegetation_ID_Ecobejct2010_CaldorVegBurnSeverity", conn)
    # columns to keep
    columns_to_keep = ['SpatialVar', 'Ownership', 'Development',
                    'WHRTYPE', 'Acres','QMD','Elev_Ft', 'SeralStage', 
                    'TRPA_VegType','gridcode','FID_CaldorBurnSeverity']
    # filter out columns
    df = df[columns_to_keep]
    # filter out Development
    df = df[df['Development'] =='Undeveloped']
    return df

In [ ]:
## Spatial Analysis to get Ecobject 2010 data 
## overlayed with Caldor Vegetation Burn Severity

# get moderate and high severity from Caldor Vegetation Burn Severity
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="CaldorBurnSeverity",
    selection_type="NEW_SELECTION",
    where_clause="gridcode IN (3, 4)",
    invert_where_clause=None
)
# overlay 2010 Ecobject data with Caldor Burn Severity
arcpy.analysis.Identity(
    in_features="Vegetation - Ecobject 2010",
    identity_features="CaldorBurnSeverity selection",
    out_feature_class=r"C:\GIS\Scratch.gdb\Identity_Ecobject2010_CaldorVegBurnModerateHigh",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)
# calculate acres for each TRPA_VegType, gridcode, SeralStage
arcpy.management.CalculateField(
    in_table="Identity_Ecobject2010_CaldorVegBurnModerateHigh",
    field="Acres",
    expression="!Shape_Area@ACRES!",
    expression_type="PYTHON3",
    code_block=None
)
# export to csv
arcpy.conversion.TableToTable(
    in_rows="Identity_Ecobject2010_CaldorVegBurnModerateHigh",
    out_name=r"C:\GIS\Scratch.gdb",
    out_location=r"C:\GIS\Scratch.gdb"
)

In [51]:
# data frame of 2010 Ecobject data 
# with Caldor Vegetation Burn Severity attributes
dfVeg = get_ecobject_caldor_identity_data()


In [131]:
# pivot Acres in df to get the sum of Acres for TRPA_VegType, gridcode, SeralStage
dfVegType      = dfVeg.pivot_table(index=['TRPA_VegType'], values='Acres', aggfunc='sum').reset_index().round(1)
dfVegSeralBurn = dfVeg.pivot_table(index=['TRPA_VegType'], columns=['gridcode', 'SeralStage'], values='Acres', aggfunc='sum').reset_index().round(1)
dfVegSeral     = dfVeg.pivot_table(index=['TRPA_VegType'], columns=['SeralStage'], values='Acres', aggfunc='sum').reset_index().round(1)
dfVegHHBurn    = dfVeg.pivot_table(index=['TRPA_VegType'], columns=['gridcode', 'SpatialVar'], values='Acres', aggfunc='sum').reset_index().round(1)
dfVegHH        = dfVeg.pivot_table(index=['TRPA_VegType'], columns=['SpatialVar'], values='Acres', aggfunc='sum').reset_index().round(1)
dfVegBurn      = dfVeg.pivot_table(index=['TRPA_VegType'], columns=['gridcode'], values='Acres', aggfunc='sum').reset_index().round(1)

In [142]:
dfVegType.rename(columns={'TRPA_VegType':"Vegetation Type"}, inplace=True)
dfVegType.to_csv(localpath.parent / '2023/data/processed_data/Ecobject_VegetationSummary.csv', index=False)
dfVegHH.rename(columns={'TRPA_VegType':"Vegetation Type"}, inplace=True)
dfVegHH.fillna(0, inplace=True)
dfVegHH.to_csv(localpath.parent / '2023/data/processed_data/Ecobject_HorizontalHeterogeneitySummary.csv', index=False)
dfVegSeral.rename(columns={'TRPA_VegType':"Vegetation Type"}, inplace=True)
dfVegSeral.fillna(0, inplace=True)
dfVegSeral.to_csv(localpath.parent / '2023/data/processed_data/Ecobject_SeralStageSummary.csv', index=False)

In [91]:
# sum Columns 3 and 4 to get Total_Burned
dfVegBurn['Total_Burned'] = dfVegBurn[3] + dfVegBurn[4]
# rename columns 
dfVegBurn.rename(columns={'TRPA_VegType':"Vegetation Type", 0:'Unburned',3:'Moderate', 4:'High'}, inplace=True)
dfVegType.rename(columns={'TRPA_VegType':"Vegetation Type"}, inplace=True)
# merge dataframes
dfNewVegSummary = pd.merge(dfVegType, dfVegBurn, on='Vegetation Type', how='left')
# fill NaN with 0
dfNewVegSummary.fillna(0, inplace=True)
# save to csv
outcsv = localpath.parent / '2023/data/processed_data/Ecobject_CaldorBurnSeveritySummary.csv'
dfNewVegSummary.to_csv(outcsv, index=False)

In [120]:
dfVegSeralBurn = dfVeg.pivot_table(index=['TRPA_VegType'], columns=['gridcode','SeralStage'], values='Acres', aggfunc='sum').reset_index().round(1)
df = dfVegSeralBurn
# rename columns
df.rename(columns={'TRPA_VegType':"Vegetation Type", 0:'Unburned',3:'Moderate', 4:'High'}, inplace=True)
# fill NaN with 0
df.fillna(0, inplace=True)
#
# flatten columns
df.columns = [' '.join(col).strip() for col in df.columns.values]
# combine open and closed canopy columns
df['Unburned Early Seral'] = df['Unburned Early Seral Closed'] + df['Unburned Early Seral Open']
df['Unburned Mid Seral'] = df['Unburned Mid Seral Closed'] + df['Unburned Mid Seral Open']
df['Unburned Late Seral'] = df['Unburned Late Seral Closed'] + df['Unburned Late Seral Open']
df['Moderate Severity Early Seral'] = df['Moderate Early Seral Closed'] + df['Moderate Early Seral Open']
df['Moderate Severity Mid Seral'] = df['Moderate Mid Seral Closed'] + df['Moderate Mid Seral Open']
df['Moderate Severity Late Seral'] = df['Moderate Late Seral Closed'] + df['Moderate Late Seral Open']
df['High Severity Early Seral'] = df['High Early Seral Closed'] + df['High Early Seral Open']
df['High Severity Mid Seral'] = df['High Mid Seral Closed'] + df['High Mid Seral Open']
df['High Severity Late Seral'] = df['High Late Seral Closed'] + df['High Late Seral Open']

# columns to keep
columns_to_keep = ['Vegetation Type',
                   'Moderate Severity Early Seral','Moderate Severity Mid Seral','Moderate Severity Late Seral',
                   'High Severity Early Seral','High Severity Mid Seral','High Severity Late Seral', 
                   'Moderate N/A', 'High N/A' ]

# filter to Red Fir Forest and Yellow Pine Forest
# df = df[df['Vegetation Type'].isin(['Red Fir Forest','Yellow Pine Forest'])]
df = df[columns_to_keep]

df.to_csv(localpath.parent / '2023/data/processed_data/Ecobject_CaldorBurnSeverity_SeralStageSummary.csv', index=False)

In [132]:
dfVegHHBurn = dfVeg.pivot_table(index=['TRPA_VegType'], columns=['gridcode', 'SpatialVar'], values='Acres', aggfunc='sum').reset_index().round(1)
df = dfVegHHBurn
# rename columns
df.rename(columns={'TRPA_VegType':"Vegetation Type", 0:'Unburned',3:'Moderate', 4:'High'}, inplace=True)
# fill NaN with 0
df.fillna(0, inplace=True)
# columns to keep
columns_to_keep = ['Vegetation Type', 'Moderate Clump',
 'Moderate Dense Clump',
 'Moderate Individual',
 'Moderate Open',
 'Moderate Scattered Clump',
 'Moderate Sparse',
 'Moderate Stand Initiation',
 'Moderate water',
 'High Clump',
 'High Dense Clump',
 'High Individual',
 'High Open',
 'High Scattered Clump',
 'High Sparse',
 'High Stand Initiation']
# flatten columns
df.columns = [' '.join(col).strip() for col in df.columns.values]

# filter to Red Fir Forest and Yellow Pine Forest
# df = df[df['Vegetation Type'].isin(['Red Fir Forest','Yellow Pine Forest'])]
df = df[columns_to_keep]
df.to_csv(localpath.parent / '2023/data/processed_data/Ecobject_CaldorBurnSeverity_HorizontalHeterogeneity.csv', index=False)

In [147]:
# set the template, font, and config for the charts
template = 'plotly_white'
font     = 'Calibri'
config   = {"displayModeBar": False}


In [ ]:
colors = ['lightslategray',] * 10
colors[1] = '#ff0000'

df = dfVeg.copy()

# df= df.loc[(df['Development']=='Undeveloped')&(df['QMD']<11)]
df= df.loc[(df['Development']=='Undeveloped')]


table = pd.pivot_table(df, values=['Acres'], index=['TRPA_VegType'],
                        aggfunc=np.sum)

flattened = pd.DataFrame(table.to_records())

flattened.columns = [hdr.replace("('Acres', '", '').replace("')", "") \
                     for hdr in flattened.columns]

df = flattened
df['TRPA_VegType'].replace('', np.nan, inplace=True)
df = df.dropna(subset=['TRPA_VegType'])

df['TotalAcres']= 171438.19
df['VegPercent'] = (df['Acres']/df['TotalAcres'])*100

df = df.sort_values('Acres', ascending=False)

# setup chart
fig = px.bar(df, y="TRPA_VegType", x='VegPercent',  color='TRPA_VegType', color_discrete_sequence=colors,
            custom_data=['Acres','TotalAcres'], orientation='h',)

fig.update_traces(
    name='',
#     hoverinfo = "y",  
    hovertemplate="<br>".join([
        "<b>%{x:.1f}%</b>",
        "or <b>%{customdata[0]:,.0f}</b> acres<br>of the %{customdata[1]:,.0f} total acres<br>of undisturbed vegetation"
    ])
)

# create coverage bars
fig.add_trace(go.Bar(
    x=[6.23],
    y=['Red Fir Forest'],
    name= "Red Fir Forest - QMD<11",
    marker_color='#FF7F7F', 
#     marker_line_color='rgb(88,48,10)',
#     opacity=0.6,
    orientation='h',
    hovertemplate='<b>%{x:,.1f}%</b><br>or <b>10,673</b> acres<br>of total undisturbed vegetation <br>is immature Red Fir forest<br>(QMD<11")<extra></extra>'
))

# create coverage bars
fig.add_trace(go.Bar(
    x=[0.5],
    y=['Red Fir Forest'],
    name= "Red Fir Forest - Burned in Caldor Fire",
    marker_color='#000000', 
#     marker_line_color='rgb(88,48,10)',
#     opacity=0.6,
    orientation='h',
    hovertemplate='<b>%{x:,.1f}%</b><br>or <b>862</b> acres<br> of total undisturbed vegetation<br>was Red Fir forest that burned in the Caldor Fire<extra></extra>'
))

# set layout
fig.update_layout(title="Relative Abundance of Red Fir Forest In Seral Stages Other Than Mature",
                    font_family=font,
                    template=template,
                    legend_title_text='',
                    showlegend=False,
                    hovermode="y unified",
                    barmode = 'overlay',
                    yaxis = dict(
                        tickmode = 'linear',
                        title_text='Vegetation Type'
                    ),
                    xaxis = dict(
                        tickmode = 'linear',
                        tick0 = 0,
                        dtick = 10,
                        ticksuffix='%',
                        range=[0, 60],
                        title_text='% of undisturbed vegetation'
                    )
                 )
fig.show()


In [ ]:
colors = ['lightslategray',] * 10
colors[9] ='#a8a800'

df = dfVeg.copy()

# df= df.loc[(df['Development']=='Undeveloped')&(df['QMD']<11)]
df= df.loc[(df['Development']=='Undeveloped')]


table = pd.pivot_table(df, values=['Acres'], index=['TRPA_VegType'],
                        aggfunc=np.sum)

flattened = pd.DataFrame(table.to_records())

flattened.columns = [hdr.replace("('Acres', '", '').replace("')", "") \
                     for hdr in flattened.columns]

df = flattened
df['TRPA_VegType'].replace('', np.nan, inplace=True)
df = df.dropna(subset=['TRPA_VegType'])

df['TotalAcres']= 171438.19
df['VegPercent'] = (df['Acres']/df['TotalAcres'])*100


# df2 = {'TRPA_VegType': 'Yellow Pine Forest', 'Acres': 23836.14, 'VegPercent': 13.9} 
# df = df.append(df2, ignore_index = True) 

# setup chart
fig = px.bar(df, x="TRPA_VegType", y='VegPercent',  color='TRPA_VegType', color_discrete_sequence=colors,
            custom_data=['Acres','TotalAcres'])

fig.update_traces(
    name='',
#     hoverinfo = "y",  
    hovertemplate="<br>".join([
        "<b>%{y:.1f}%</b>",
        "or <b>%{customdata[0]:,.0f}</b> acres<br>of the %{customdata[1]:,.0f} total acres<br>of undisturbed vegetation"
    ])
)

# create coverage bars
fig.add_trace(go.Bar(
    y=[13.9],
    x=['Yellow Pine Forest'],
    name= "Yellow Pine Forest - QMD<11",
    marker_color='#ffffbf', 
    marker_line_color='rgb(88,48,10)',
    opacity=0.6,
    hovertemplate='<b>%{y:,.1f}%</b><br>or <b>23,836</b> acres<br>of immature forest<br>(QMD<11")<extra></extra>'
))

# set layout
fig.update_layout(title="Relative Abundance of Yellow Pine Forest In Seral Stages Other Than Mature",
                    font_family=font,
                    template=template,
                    legend_title_text='',
                    showlegend=False,
                    hovermode="x unified",
                    barmode = 'overlay',
                    xaxis = dict(
#                         categoryorder= 'array',
#                         categoryarray= ['Early Seral', 'Mid Seral', 'Late Seral', 'N/A'],
                        tickmode = 'linear',
                        title_text='Vegetation Type'
                    ),
                    yaxis = dict(
                        tickmode = 'linear',
                        tick0 = 0,
                        dtick = 10,
                        ticksuffix='%',
                        range=[0, 60],
                        title_text='% of undisturbed vegetation'
                    )
                 )

fig.show()